In [ ]:
# Imports and installations
%pip install transformers[torch]

from transformers import pipeline
from google.colab import drive
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
import json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Use CUDA (GPU)
    device = torch.device("cuda")
    print('GPU:', torch.cuda.get_device_name(0))
else:
    # Use CPU
    device = torch.device("cpu")
    print('Using CPU.')

GPU: NVIDIA L4


In [ ]:
# Mount drive
drive.mount("/content/drive")

In [ ]:
# Initialize BART tokenizer and configuration from finetuned model
tokenizer = BartTokenizer.from_pretrained("/content/drive/MyDrive/cpsc_477/training_2/checkpoint-9000")
config = BartConfig.from_pretrained("/content/drive/MyDrive/cpsc_477/training_2/checkpoint-9000")
# Define model using loaded configuration
model = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/cpsc_477/training_2/checkpoint-9000", config=config)

In [ ]:
# Extract articles from dataset
def get_articles():
	# Define path to jsonl file
	jsonl_file_path = "/content/drive/MyDrive/cpsc_477/data/copy_PLOS_val.jsonl"
	# read file
	with open(jsonl_file_path, "r", encoding="utf-8") as f:
			articles = []
			for line in f:
					# Load json object on line
					article_object = json.loads(line)
					# just extract article from jsno object
					article_text = article_object['article']
					articles.append(article_text)
	return articles

In [ ]:
# Write summaries to a file for inference, one per line
def write_summaries_to_file(summaries, output_file_path):
    """
    Write summaries to a text file.

    Args:
    - summaries (list): List of summaries.
    - output_file_path (str): Path to the output text file.
    """
    with open(output_file_path, "w", encoding="utf-8") as f:
        for summary in summaries:
            f.write(summary.strip() + "\n")

In [ ]:
# Main code to extract summaries from our model

content = get_articles() # load in articles
summaries = []
for text in content:
  inputs = tokenizer.encode(text, max_length=1024, truncation=True, return_tensors='pt') # tokenize input
  pred = model.generate(inputs, max_length=300, min_length=100, num_beams=4, early_stopping=True) # generate summary
  summary = tokenizer.decode(pred[0], skip_special_tokens=True) # decode generated summary tokens into text
  # Check if summary is blank, if so model failed
  # Never happened for BART though
  if summary == "" or summary == " ":
    summary = "None."
  summaries.append(summary)

# Once we have all summaries, write them to a text file
write_summaries_to_file(summaries, "/content/drive/MyDrive/cpsc_477/data/bart_summaries.txt")